#### Feature Engineering 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

#### Categorical features

In [2]:
df = pd.read_csv('Churn_Banking_Modeling.csv')

for col in df.columns:
    if col.startswith('Flag'):
        df[col] = df[col].astype('object')


numeric_variables_count = len(df.select_dtypes(include=['number']).columns)
categorical_variables_count = len(df.select_dtypes(include=['object']).columns)

print("number of numeric features",numeric_variables_count)
print("number of categorical features",categorical_variables_count)

print(df.describe(include=['object']))

FileNotFoundError: [Errno 2] No such file or directory: 'Churn_Banking_Modeling.csv'

###### The features 'Professione' has two same modalities, so we merged them.

In [ ]:
df_new=df
df_new['Professione'] = df_new['Professione'].apply(lambda x: 'Ufficiale/Sottoufficiale' if x == 'Ufficiale/Sottufficiale' else 
                                                                  'Libero professionista/Titolare impresa' if x == 'Libero professionista/Titolare di impresa' else 
                                                                  x)
df_new['Professione'].unique()

array(['Impiegato', 'Altro/Nessuno', 'Quadro', 'Operaio', nan,
       'Ufficiale/Sottoufficiale',
       'Libero professionista/Titolare impresa', 'Dirigente',
       'Commercialista', 'Rappresentante/Agente', 'Ingegnere/Architetto',
       'Avvocato', 'Medico/Farmacista', 'Autotrasportatore',
       'Docente/Insegnante', 'Commesso/Ausiliario', 'Notaio'],
      dtype=object)

###### The features Provincia_Residenza and Provincia_Domicilio have different modalities referred to the same place, for example, for Forlì-Cesena exist FO and FC, so we merged them. 

In [ ]:
domicilio_counts = df_new['Provincia_Domicilio'].value_counts()
residenza_counts = df_new['Provincia_Residenza'].value_counts()

print(domicilio_counts[domicilio_counts<100], residenza_counts[residenza_counts<100])

frequenze_PU = df_new[df_new['Provincia_Domicilio'] == 'PU'].shape[0]
print("Absolute frequencies of 'PU':", frequenze_PU)

frequenze_FC = df_new[df_new['Provincia_Domicilio'] == 'FC'].shape[0]
print("Absolute frequencies of 'FC':", frequenze_FC)

Provincia_Domicilio
OG    51
FO    29
PS    26
8N     1
BE     1
Name: count, dtype: int64 Provincia_Residenza
OG    69
PS     2
FO     1
Name: count, dtype: int64
Absolute frequencies of 'PU': 2412
Absolute frequencies of 'FC': 2099


In [ ]:
df_new['Provincia_Residenza'] = df_new['Provincia_Residenza'].replace('FO', 'FC')
df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('FO', 'FC')

df_new['Provincia_Residenza'] = df_new['Provincia_Residenza'].replace('PS', 'PU')
df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('PS', 'PU')

df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('8N', np.nan)
df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('BE', np.nan)

domicilio_counts = df_new['Provincia_Domicilio'].value_counts()
residenza_counts = df_new['Provincia_Residenza'].value_counts()

print(domicilio_counts[domicilio_counts<100], residenza_counts[residenza_counts<100])

Provincia_Domicilio
OG    51
Name: count, dtype: int64 Provincia_Residenza
OG    69
Name: count, dtype: int64


###### Insted of changing all categorical features in dummies, there are some ordinal categorical features that we can encode.

In [ ]:
mapping = {
    'CAUTO': 1,
    'PRUDENTE': 2,
    'BILANCIATO': 3,
    'DINAMICO':4,
    'ND':0
}
df_new['Profilo_MIFID'] = df_new['Profilo_MIFID'].replace(mapping)

mapping2 = {
    'Bassissimo (<1200)': 1,
    'Basso (tra 1200 e 1500)': 2,
    'Medio (tra 1500 e 1800)': 3,
    'Alto (tra 1800 e 2500)':4,
    'Altissimo (>2500)':5
}

df_new['Imp_Reddito'] = df_new['Imp_Reddito'].replace(mapping2)

#### Numerical features

In [ ]:
df_new.describe(include=['int64','float64'])

,Id_Cliente,Imp_Valore_del_Cliente,Eta,Anno_Apertura_primo_Conto,Imp_Reddito,Profilo_MIFID,Imp_Canone_Pricing_cc,Imp_Bonifici_vs_Competitors,Imp_Bonifici_vs_No_Competitors,Num_Utenze_in_essere,...,Num_attivit�_trading,Var_Numero_Utenze,Iscrizione_programma_Loyalty,Num_Punti_Programma_Loyalty,Imp_Liquidit�_Attuale,Imp_Gestito_attuale,Imp_Amministrato_attuale,Imp_Liquidit�_Attuale_6m,Imp_Gestito_attuale_6m,Imp_Amministrato_attuale_6m
count,377369.000000,377369.000000,377362.000000,377369.000000,157476.000000,328112.000000,349973.000000,8.871400e+04,2.006900e+05,209937.000000,...,86931.000000,218355.000000,377369.000000,127239.000000,3.773640e+05,1.111880e+05,1.876340e+05,3.773560e+05,1.109000e+05,1.862980e+05
mean,277680.735847,644.662654,46.039890,2004.805750,3.209340,2.550681,0.903291,3.183130e+03,3.352734e+03,4.454517,...,15.143344,0.198044,0.337174,1045.617303,1.513478e+04,6.770461e+04,7.614477e+04,1.532084e+04,6.554481e+04,7.233862e+04
std,160279.531097,2928.725757,13.146689,3.288509,1.369838,1.451032,1.825685,1.423111e+04,1.511987e+04,3.636854,...,78.764646,1.071767,0.472745,3515.585204,5.204623e+04,1.940914e+05,2.669638e+05,5.201275e+04,1.780930e+05,2.637481e+05
min,2.000000,0.000000,3.000000,1994.000000,1.000000,0.000000,0.000000,1.000000e-02,1.000000e-02,1.000000,...,1.000000,-26.000000,0.000000,-50.000000,-2.998609e+06,0.000000e+00,0.000000e+00,-2.990061e+06,0.000000e+00,0.000000e+00
25%,138834.000000,46.870000,37.000000,2002.000000,2.000000,2.000000,0.000000,1.575825e+02,3.500000e+02,2.000000,...,1.000000,0.000000,0.000000,150.000000,6.292100e+02,7.880375e+03,4.966840e+03,6.313775e+02,7.808850e+03,4.937938e+03
50%,277965.000000,97.270000,43.000000,2005.000000,3.000000,3.000000,0.000000,4.790000e+02,8.058555e+02,3.000000,...,2.000000,0.000000,0.000000,495.000000,3.850860e+03,2.329358e+04,2.084153e+04,3.788385e+03,2.284953e+04,2.016169e+04
75%,416378.000000,370.320000,54.000000,2008.000000,4.000000,4.000000,0.325000,1.500000e+03,2.047996e+03,6.000000,...,6.000000,0.000000,1.000000,1065.000000,1.417872e+04,6.469545e+04,6.766876e+04,1.397786e+04,6.297351e+04,6.434840e+04
max,555150.000000,377934.870000,132.000000,2010.000000,5.000000,4.000000,5.950000,1.100000e+06,1.130700e+06,40.000000,...,6812.000000,25.000000,1.000000,427530.000000,1.686020e+07,1.736378e+07,4.808764e+07,1.486619e+07,1.838148e+07,5.715864e+07


#### Dropping of some features


In [ ]:
df_new = df_new.drop(['Imp_Gestito_attuale_6m', 'Imp_Liquidit�_Attuale_6m', 'Imp_Amministrato_attuale_6m', 'Id_Cliente'], axis =1)
df_new.head()

,Flag_Richiesta_Estinzione_cc,Imp_Valore_del_Cliente,Flag_Apertura_Conto_Online,Flag_Possesso_piu_Conti,Eta,Provincia_Domicilio,Provincia_Residenza,Anno_Apertura_primo_Conto,Professione,Imp_Reddito,...,Flag_Contatto_Call_Center,Flag_variazione_Accredito_Stipendio,Num_Punti_Programma_Loyalty,Imp_Liquidit�_Attuale,Imp_Gestito_attuale,Imp_Amministrato_attuale,Flag_Trasferimento_Titoli_Out,Flag_Rifiuto_Carte,Flag_Rifiuto_Prestiti,Flag_Disattivazione_RID
0,no,65.03,0,0,38.0,BO,BO,2004.0,Impiegato,3.0,...,1.0,0.0,NaN,1634.57,2978.40,2980.92,0,0,0,0
1,no,138.88,0,0,45.0,SA,SA,2000.0,Impiegato,4.0,...,1.0,-1.0,325.0,11918.26,NaN,34916.15,0,0,0,0
2,no,546.54,0,0,61.0,VT,VT,2005.0,Altro/Nessuno,4.0,...,1.0,0.0,460.0,2671.95,NaN,232776.62,1,0,0,0
3,no,68.69,0,0,33.0,MI,MI,2010.0,Impiegato,2.0,...,0.0,-1.0,NaN,19211.31,15013.53,15017.53,0,0,0,0
4,no,2417.05,0,0,36.0,MI,MI,2001.0,Quadro,NaN,...,1.0,NaN,3650.0,1526.38,NaN,NaN,0,0,0,0


#### Creation of the new dataset with the changes applied

In [ ]:
#df_model.to_csv("Churn_clean.csv", index=False)
df_new.to_csv("Churn_clean_competition.csv", index=False)